In [6]:
from __future__ import print_function
import argparse
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from dotmap import DotMap


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args=DotMap()
args.batch_size=128
args.saveInt=1
args.epochs=20
args.cuda=True
args.seed=1
args.log_interval=10
args.outf='./vae_mnist'

args.cuda = not args.no_cuda and torch.cuda.is_available()
try:
    os.makedirs(args.outf)
except OSError:
    pass


torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)



kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
          std = logvar.mul(0.5).exp_()
          eps = Variable(std.data.new(std.size()).normal_())
          return eps.mul(std).add_(mu)
        else:
          return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
if args.cuda:
    model.cuda()


def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # Normalise by same number of elements as in reconstruction
    KLD /= args.batch_size * 784

    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        if args.cuda:
            data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        if args.cuda:
            data = data.cuda()
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]
        if i == 0:
          n = min(data.size(0), 8)
          comparison = torch.cat([data[:n],
                                  recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
          save_image(comparison.data.cpu(),
                     args.outf+'/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)
    sample = Variable(torch.randn(64, 20))
    if args.cuda:
       sample = sample.cuda()
    sample = model.decode(sample).cpu()
    save_image(sample.data.view(64, 1, 28, 28),
               args.outf+'/sample_' + str(epoch) + '.png')

    if epoch%args.saveInt == 0 and epoch!=0:
        torch.save(model.state_dict(), '%s/vae_epoch_%d.pth' % (args.outf, epoch))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.005474
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.003104
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.002357
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.002247
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.002083
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.002059
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.002080
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.002007
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.001911
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.001949
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.001897
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.001726
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.001784
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.001689
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.001682
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.001616
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.001573
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.001578
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.001549
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.001577
Train Epoch: 1 [

Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.001151
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.001152
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.001109
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.001082
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.001092
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.001084
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.001149
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.001092
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.001089
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.001055
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.001124
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.001135
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.001110
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.001101
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.001147
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.001128
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.001085
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.001123
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.001103
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.001098


Train Epoch: 7 [52480/60000 (87%)]	Loss: 0.001098
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.001067
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.001054
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.001063
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.001076
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.001044
====> Epoch: 7 Average loss: 0.0011
====> Test set loss: 0.0010
Train Epoch: 8 [0/60000 (0%)]	Loss: 0.001073
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.001034
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.001048
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.001070
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.001036
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.001063
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.001054
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.001061
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.001068
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.001055
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.001124
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.001033
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.001028
Tr

Train Epoch: 11 [15360/60000 (26%)]	Loss: 0.001053
Train Epoch: 11 [16640/60000 (28%)]	Loss: 0.001055
Train Epoch: 11 [17920/60000 (30%)]	Loss: 0.001070
Train Epoch: 11 [19200/60000 (32%)]	Loss: 0.001049
Train Epoch: 11 [20480/60000 (34%)]	Loss: 0.001055
Train Epoch: 11 [21760/60000 (36%)]	Loss: 0.001020
Train Epoch: 11 [23040/60000 (38%)]	Loss: 0.001064
Train Epoch: 11 [24320/60000 (41%)]	Loss: 0.001039
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.001064
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.001090
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.001064
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.001052
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.001042
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.001072
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.001033
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.001059
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.001059
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.001059
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.001057
Train Epoch: 11 [39680/60000 (6

Train Epoch: 14 [37120/60000 (62%)]	Loss: 0.001045
Train Epoch: 14 [38400/60000 (64%)]	Loss: 0.001033
Train Epoch: 14 [39680/60000 (66%)]	Loss: 0.001066
Train Epoch: 14 [40960/60000 (68%)]	Loss: 0.001076
Train Epoch: 14 [42240/60000 (70%)]	Loss: 0.001036
Train Epoch: 14 [43520/60000 (72%)]	Loss: 0.001022
Train Epoch: 14 [44800/60000 (75%)]	Loss: 0.001106
Train Epoch: 14 [46080/60000 (77%)]	Loss: 0.001059
Train Epoch: 14 [47360/60000 (79%)]	Loss: 0.001061
Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.001029
Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.001030
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.001054
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.001069
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.001030
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.001034
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.001014
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.001023
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.001033
====> Epoch: 14 Average loss: 0.0010
====> Test set loss: 0.0010
Train Epoch: 15 [

Train Epoch: 17 [58880/60000 (98%)]	Loss: 0.001024
====> Epoch: 17 Average loss: 0.0010
====> Test set loss: 0.0010
Train Epoch: 18 [0/60000 (0%)]	Loss: 0.001060
Train Epoch: 18 [1280/60000 (2%)]	Loss: 0.001051
Train Epoch: 18 [2560/60000 (4%)]	Loss: 0.001055
Train Epoch: 18 [3840/60000 (6%)]	Loss: 0.001031
Train Epoch: 18 [5120/60000 (9%)]	Loss: 0.001028
Train Epoch: 18 [6400/60000 (11%)]	Loss: 0.001059
Train Epoch: 18 [7680/60000 (13%)]	Loss: 0.001040
Train Epoch: 18 [8960/60000 (15%)]	Loss: 0.001021
Train Epoch: 18 [10240/60000 (17%)]	Loss: 0.001032
Train Epoch: 18 [11520/60000 (19%)]	Loss: 0.001041
Train Epoch: 18 [12800/60000 (21%)]	Loss: 0.001024
Train Epoch: 18 [14080/60000 (23%)]	Loss: 0.001021
Train Epoch: 18 [15360/60000 (26%)]	Loss: 0.001026
Train Epoch: 18 [16640/60000 (28%)]	Loss: 0.001038
Train Epoch: 18 [17920/60000 (30%)]	Loss: 0.001025
Train Epoch: 18 [19200/60000 (32%)]	Loss: 0.001023
Train Epoch: 18 [20480/60000 (34%)]	Loss: 0.001024
Train Epoch: 18 [21760/60000 (36%

In [16]:
# train classifier
import torch.nn.functional as F

class Net_Classifier(nn.Module):
    def __init__(self):
        super(Net_Classifier, self).__init__()
        self.fc1 = nn.Linear(20, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.log_softmax(x)

net_classifier=Net_Classifier()
if args.cuda:
    net_classifier.cuda()

optimizer_classifier = optim.Adam(net_classifier.parameters(), lr=1e-3)    
def train_classifier(epoch):
    model.eval()
    net_classifier.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer_classifier.zero_grad()
        recon_batch, mu, logvar = model(data)
        output = net_classifier(mu)
        train_loss = F.nll_loss(output, target)
        train_loss.backward()
        optimizer_classifier.step()
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss.data[0] / len(train_loader.dataset)))
    
def test_classifier():
    net_classifier.eval()
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        recon_batch, mu, logvar = model(data)
        output = net_classifier(mu)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, args.epochs + 1):
    train_classifier(epoch)
    test_classifier()


====> Epoch: 1 Average loss: 0.0000

Test set: Average loss: 0.2842, Accuracy: 9195/10000 (92%)

====> Epoch: 2 Average loss: 0.0000

Test set: Average loss: 0.2017, Accuracy: 9419/10000 (94%)

====> Epoch: 3 Average loss: 0.0000

Test set: Average loss: 0.1612, Accuracy: 9534/10000 (95%)

====> Epoch: 4 Average loss: 0.0000

Test set: Average loss: 0.1387, Accuracy: 9585/10000 (96%)

====> Epoch: 5 Average loss: 0.0000

Test set: Average loss: 0.1209, Accuracy: 9633/10000 (96%)

====> Epoch: 6 Average loss: 0.0000

Test set: Average loss: 0.1121, Accuracy: 9653/10000 (97%)

====> Epoch: 7 Average loss: 0.0000

Test set: Average loss: 0.1051, Accuracy: 9676/10000 (97%)

====> Epoch: 8 Average loss: 0.0000

Test set: Average loss: 0.0975, Accuracy: 9704/10000 (97%)

====> Epoch: 9 Average loss: 0.0000

Test set: Average loss: 0.0943, Accuracy: 9710/10000 (97%)

====> Epoch: 10 Average loss: 0.0000

Test set: Average loss: 0.0927, Accuracy: 9722/10000 (97%)

====> Epoch: 11 Average loss:

In [114]:
import random
import numpy as np
for data,target in test_loader:
    model.eval()
    if args.cuda:
        data, target = data[0].unsqueeze(1).cuda(), target.unsqueeze(1)[0].cuda()
    data, target = Variable(data, volatile=True), Variable(target)
    recon_batch, mu, logvar = model(data)
    output = net_classifier(mu)
    pred = output.data.max(1, keepdim=True)
    res=pred[1].cpu()[0][0]
    adv_targets=set(list(range(0,10)))-set([res])
    adv_target=random.sample(adv_targets, 1)[0]
    #print(output)
    
    
    # Generate Adversarial Image 
    x = Variable(mu.data, requires_grad=True)
    y_true=target
    SoftmaxWithXent = nn.CrossEntropyLoss()
    # Forward pass
    outputs = net_classifier(x)
    loss = SoftmaxWithXent(outputs, y_true)
    loss.backward() # obtain gradients on x

    # Add perturbation
    epsilon = 0.1 
    x_grad   = torch.sign(x.grad.data)
    x_adversarial = torch.clamp(x.data + epsilon * x_grad, -0.5, 0.5) 

    # Classification after optimization  
    y_pred_adversarial = np.argmax(net_classifier(Variable(x_adversarial)).cpu().data.numpy())

    print(y_true)
    print(res)
    print(y_pred_adversarial)
    x_adversarial=x_adversarial.cuda()
    adv_z = Variable(x_adversarial)
    
    adv_recon_batch=model.decode(adv_z)
    
    save_image(data.data.cpu(),'test.png')
    save_image(recon_batch.view(1,28,28).data.cpu(),'recon.png')
    save_image(adv_recon_batch.view(1,28,28).data.cpu(),'adv_recon.png')
    break


Variable containing:
 6
[torch.cuda.LongTensor of size 1 (GPU 0)]

6
3


In [117]:
from IPython.display import Image
Image("test.png")

In [118]:
Image('recon.png')

In [119]:
Image('adv_recon.png')